In [1]:
# Credits: https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
# LSTM for sequence classification in the IMDB dataset
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import seaborn as sns
import keras
from keras.preprocessing.text import Tokenizer
import sqlite3
from tqdm import tqdm
import re
from bs4 import BeautifulSoup

from sklearn.model_selection import train_test_split
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)


Using TensorFlow backend.


In [2]:
conn = sqlite3.connect('C:/Users/pramod reddy chandi/Desktop/pram/applied ai course/AFR_2018/database.sqlite')
filtered_data = pd.read_sql_query(''' SELECT * FROM REVIEWS LIMIT 50000''', conn)

# Give reviews with Score>3 a positive rating(1), and reviews with a score<3 a negative rating(0).
def partition(x):
    if x < 3:
        return 0
    return 1

def findMinorClassPoints(df):
    posCount = int(df[df['Score']==1].shape[0]);
    negCount = int(df[df['Score']==0].shape[0]);
    if negCount < posCount:
        return negCount
    return posCount

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative

#Performing Downsampling
# samplingCount = findMinorClassPoints(filtered_data)
# postive_df = filtered_data[filtered_data['Score'] == 1].sample(n=5000)
# negative_df = filtered_data[filtered_data['Score'] == 0].sample(n=5000)

# filtered_data = pd.concat([postive_df, negative_df])

print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (50000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than most.
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,"Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as ""Jumbo""."
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all","This is a confection that has been around a few centuries. It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar. And it is a tiny mouthful of heaven. Not too chewy, and very flavorful. I highly recommend this yummy treat. If you are familiar with the story of C.S. Lewis' ""The Lion, The Witch, and The Wardrobe"" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch."


# data preprocessing

In [3]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

#Removing the anamolies
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

#Preprocessing
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [4]:
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(final['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    # sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())
    
## Similartly you can do preprocessing for review summary also.
def concatenateSummaryWithText(str1, str2):
    return str1 + ' ' + str2

preprocessed_summary = []
# tqdm is for printing the status bar
for sentence in tqdm(final['Summary'].values):
    sentence = re.sub(r"http\S+", "", sentence)
    #sentence = BeautifulSoup(sentence, 'lxml').get_text()
    sentence = decontracted(sentence)
    sentence = re.sub("\S*\d\S*", "", sentence).strip()
    sentence = re.sub('[^A-Za-z]+', ' ', sentence)
    # https://gist.github.com/sebleier/554280
    # sentence = ' '.join(e.lower() for e in sentence.split() if e.lower() not in stopwords)
    preprocessed_summary.append(sentence.strip())
    
preprocessed_reviews = list(map(concatenateSummaryWithText, preprocessed_reviews, preprocessed_summary))
final['CleanedText'] = preprocessed_reviews
final['CleanedText'] = final['CleanedText'].astype('str')

100%|██████████| 46267/46267 [00:01<00:00, 24598.59it/s]


In [5]:
X = final['CleanedText']
y = final['Score']

In [6]:
X.shape

(46267,)

In [7]:
#splitting the data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, shuffle=True)

In [8]:
X_train.shape

(34700,)

In [9]:
X_test.shape

(11567,)

In [10]:
#tokenize the data set

tokenize = Tokenizer(num_words=5000)
tokenize.fit_on_texts(X_train)

X_train_new = tokenize.texts_to_sequences(X_train)
X_test_new = tokenize.texts_to_sequences(X_test)

print(X_train_new[1])
print(len(X_train_new))

[2, 39, 10, 157, 7, 146, 311, 101, 16, 2, 84, 27, 126, 11, 13, 264, 21, 18, 44, 1159, 7, 29, 3, 1031, 11, 29, 99, 700, 1, 75, 126, 18, 1, 179, 22, 1, 203, 126, 16, 33, 4, 395, 77, 2, 20, 1445, 14, 21, 42, 30, 622, 34, 159, 138, 21, 18, 690, 11, 121, 16, 10, 25, 2, 146, 237, 203, 67, 126, 277, 600, 1, 68, 576, 6, 1070, 150, 69, 623, 37, 17, 60, 13, 264, 2285, 18, 675, 431, 3, 3147, 3, 13, 608, 615, 3078, 14, 96, 231, 5, 402, 30, 1, 126, 40, 17, 18, 233, 11, 4, 156, 806, 51, 5, 349, 7, 71, 2262, 359, 8, 1, 1032, 2047, 14, 53, 3327, 189, 12, 5, 60, 27, 84, 27, 11, 1, 264]
34700


In [11]:
# truncate and/or pad input sequences
max_review_length = 600
X_train = sequence.pad_sequences(X_train_new, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test_new, maxlen=max_review_length)


In [12]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(5001, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 600, 32)           160032    
_________________________________________________________________
lstm_1 (LSTM)                (None, 600, 100)          53200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 293,733
Trainable params: 293,733
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
model.fit(X_train, y_train, nb_epoch=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Instructions for updating:
Use tf.cast instead.
Epoch 1/10


13056/34700 [==========>...................] - ETA: 50:36 - loss: 0.6934 - acc: 0.42 - ETA: 34:40 - loss: 0.6899 - acc: 0.64 - ETA: 29:22 - loss: 0.6870 - acc: 0.68 - ETA: 27:00 - loss: 0.6835 - acc: 0.70 - ETA: 25:23 - loss: 0.6757 - acc: 0.75 - ETA: 24:20 - loss: 0.6691 - acc: 0.76 - ETA: 23:35 - loss: 0.6574 - acc: 0.78 - ETA: 23:07 - loss: 0.6458 - acc: 0.79 - ETA: 22:43 - loss: 0.6354 - acc: 0.78 - ETA: 22:20 - loss: 0.6203 - acc: 0.79 - ETA: 22:07 - loss: 0.6034 - acc: 0.79 - ETA: 21:53 - loss: 0.6026 - acc: 0.79 - ETA: 21:40 - loss: 0.5850 - acc: 0.80 - ETA: 21:29 - loss: 0.5704 - acc: 0.80 - ETA: 21:20 - loss: 0.5550 - acc: 0.81 - ETA: 21:15 - loss: 0.5433 - acc: 0.82 - ETA: 21:05 - loss: 0.5409 - acc: 0.81 - ETA: 21:00 - loss: 0.5361 - acc: 0.81 - ETA: 20:56 - loss: 0.5280 - acc: 0.82 - ETA: 20:57 - loss: 0.5204 - acc: 0.82 - ETA: 21:05 - loss: 0.5212 - acc: 0.82 - ETA: 21:10 - loss: 0.5159 - acc: 0.82 - ETA: 21:13 - loss: 0.5108 - acc: 0.82 - ETA: 21:13 - loss: 0.5062 - acc: 

26112/34700 [=====================>........] - ETA: 13:44 - loss: 0.3211 - acc: 0.87 - ETA: 13:41 - loss: 0.3209 - acc: 0.87 - ETA: 13:39 - loss: 0.3204 - acc: 0.87 - ETA: 13:37 - loss: 0.3199 - acc: 0.87 - ETA: 13:34 - loss: 0.3194 - acc: 0.87 - ETA: 13:32 - loss: 0.3190 - acc: 0.87 - ETA: 13:29 - loss: 0.3182 - acc: 0.87 - ETA: 13:27 - loss: 0.3175 - acc: 0.87 - ETA: 13:24 - loss: 0.3171 - acc: 0.87 - ETA: 13:22 - loss: 0.3174 - acc: 0.87 - ETA: 13:20 - loss: 0.3166 - acc: 0.87 - ETA: 13:17 - loss: 0.3159 - acc: 0.87 - ETA: 13:15 - loss: 0.3157 - acc: 0.87 - ETA: 13:12 - loss: 0.3153 - acc: 0.87 - ETA: 13:10 - loss: 0.3143 - acc: 0.87 - ETA: 13:07 - loss: 0.3136 - acc: 0.87 - ETA: 13:05 - loss: 0.3133 - acc: 0.87 - ETA: 13:03 - loss: 0.3130 - acc: 0.87 - ETA: 13:00 - loss: 0.3131 - acc: 0.87 - ETA: 12:58 - loss: 0.3127 - acc: 0.87 - ETA: 12:56 - loss: 0.3125 - acc: 0.87 - ETA: 12:53 - loss: 0.3118 - acc: 0.87 - ETA: 12:51 - loss: 0.3113 - acc: 0.87 - ETA: 12:48 - loss: 0.3107 - acc: 

34700/34700 [==============================] - ETA: 5:27 - loss: 0.2707 - acc: 0.890 - ETA: 5:24 - loss: 0.2707 - acc: 0.890 - ETA: 5:22 - loss: 0.2706 - acc: 0.890 - ETA: 5:19 - loss: 0.2702 - acc: 0.891 - ETA: 5:17 - loss: 0.2699 - acc: 0.891 - ETA: 5:14 - loss: 0.2696 - acc: 0.891 - ETA: 5:12 - loss: 0.2695 - acc: 0.891 - ETA: 5:10 - loss: 0.2697 - acc: 0.891 - ETA: 5:07 - loss: 0.2695 - acc: 0.891 - ETA: 5:05 - loss: 0.2695 - acc: 0.891 - ETA: 5:02 - loss: 0.2691 - acc: 0.891 - ETA: 5:00 - loss: 0.2691 - acc: 0.891 - ETA: 4:57 - loss: 0.2688 - acc: 0.891 - ETA: 4:55 - loss: 0.2686 - acc: 0.891 - ETA: 4:53 - loss: 0.2685 - acc: 0.891 - ETA: 4:50 - loss: 0.2681 - acc: 0.892 - ETA: 4:48 - loss: 0.2679 - acc: 0.892 - ETA: 4:45 - loss: 0.2677 - acc: 0.892 - ETA: 4:43 - loss: 0.2674 - acc: 0.892 - ETA: 4:40 - loss: 0.2673 - acc: 0.892 - ETA: 4:38 - loss: 0.2671 - acc: 0.892 - ETA: 4:36 - loss: 0.2668 - acc: 0.892 - ETA: 4:33 - loss: 0.2669 - acc: 0.892 - ETA: 4:31 - loss: 0.2671 - acc: 0

...

26112/34700 [=====================>........] - ETA: 7:20 - loss: 0.0477 - acc: 0.982 - ETA: 7:19 - loss: 0.0477 - acc: 0.982 - ETA: 7:18 - loss: 0.0479 - acc: 0.982 - ETA: 7:17 - loss: 0.0480 - acc: 0.982 - ETA: 7:15 - loss: 0.0479 - acc: 0.982 - ETA: 7:14 - loss: 0.0478 - acc: 0.982 - ETA: 7:12 - loss: 0.0477 - acc: 0.982 - ETA: 7:11 - loss: 0.0490 - acc: 0.981 - ETA: 7:10 - loss: 0.0489 - acc: 0.982 - ETA: 7:08 - loss: 0.0488 - acc: 0.982 - ETA: 7:07 - loss: 0.0491 - acc: 0.981 - ETA: 7:06 - loss: 0.0490 - acc: 0.981 - ETA: 7:04 - loss: 0.0492 - acc: 0.981 - ETA: 7:03 - loss: 0.0491 - acc: 0.981 - ETA: 7:02 - loss: 0.0490 - acc: 0.982 - ETA: 7:00 - loss: 0.0491 - acc: 0.981 - ETA: 6:59 - loss: 0.0489 - acc: 0.982 - ETA: 6:58 - loss: 0.0488 - acc: 0.982 - ETA: 6:57 - loss: 0.0487 - acc: 0.982 - ETA: 6:55 - loss: 0.0487 - acc: 0.982 - ETA: 6:54 - loss: 0.0490 - acc: 0.982 - ETA: 6:52 - loss: 0.0490 - acc: 0.982 - ETA: 6:51 - loss: 0.0491 - acc: 0.982 - ETA: 6:50 - loss: 0.0492 - acc: 0

34700/34700 [==============================] - ETA: 2:55 - loss: 0.0561 - acc: 0.979 - ETA: 2:53 - loss: 0.0561 - acc: 0.979 - ETA: 2:52 - loss: 0.0561 - acc: 0.979 - ETA: 2:51 - loss: 0.0560 - acc: 0.979 - ETA: 2:49 - loss: 0.0559 - acc: 0.979 - ETA: 2:48 - loss: 0.0559 - acc: 0.980 - ETA: 2:47 - loss: 0.0560 - acc: 0.979 - ETA: 2:46 - loss: 0.0561 - acc: 0.979 - ETA: 2:44 - loss: 0.0561 - acc: 0.979 - ETA: 2:43 - loss: 0.0561 - acc: 0.979 - ETA: 2:42 - loss: 0.0562 - acc: 0.979 - ETA: 2:40 - loss: 0.0561 - acc: 0.979 - ETA: 2:39 - loss: 0.0563 - acc: 0.979 - ETA: 2:38 - loss: 0.0565 - acc: 0.979 - ETA: 2:36 - loss: 0.0566 - acc: 0.979 - ETA: 2:35 - loss: 0.0565 - acc: 0.979 - ETA: 2:34 - loss: 0.0566 - acc: 0.979 - ETA: 2:32 - loss: 0.0566 - acc: 0.979 - ETA: 2:31 - loss: 0.0565 - acc: 0.979 - ETA: 2:30 - loss: 0.0564 - acc: 0.979 - ETA: 2:28 - loss: 0.0564 - acc: 0.979 - ETA: 2:27 - loss: 0.0563 - acc: 0.979 - ETA: 2:26 - loss: 0.0563 - acc: 0.979 - ETA: 2:25 - loss: 0.0563 - acc: 0

13056/34700 [==========>...................] - ETA: 11:32 - loss: 0.0786 - acc: 0.98 - ETA: 11:18 - loss: 0.1043 - acc: 0.96 - ETA: 11:13 - loss: 0.1000 - acc: 0.96 - ETA: 11:21 - loss: 0.0999 - acc: 0.96 - ETA: 11:22 - loss: 0.0873 - acc: 0.96 - ETA: 11:24 - loss: 0.0802 - acc: 0.97 - ETA: 11:27 - loss: 0.0707 - acc: 0.97 - ETA: 11:23 - loss: 0.0663 - acc: 0.97 - ETA: 11:23 - loss: 0.0611 - acc: 0.97 - ETA: 11:24 - loss: 0.0565 - acc: 0.98 - ETA: 11:24 - loss: 0.0538 - acc: 0.98 - ETA: 11:23 - loss: 0.0538 - acc: 0.98 - ETA: 11:22 - loss: 0.0539 - acc: 0.98 - ETA: 11:23 - loss: 0.0561 - acc: 0.97 - ETA: 11:26 - loss: 0.0577 - acc: 0.97 - ETA: 11:28 - loss: 0.0548 - acc: 0.97 - ETA: 11:32 - loss: 0.0540 - acc: 0.97 - ETA: 11:30 - loss: 0.0528 - acc: 0.98 - ETA: 11:29 - loss: 0.0558 - acc: 0.97 - ETA: 11:27 - loss: 0.0542 - acc: 0.97 - ETA: 11:24 - loss: 0.0545 - acc: 0.97 - ETA: 11:24 - loss: 0.0550 - acc: 0.97 - ETA: 11:24 - loss: 0.0537 - acc: 0.97 - ETA: 11:23 - loss: 0.0528 - acc: 

26112/34700 [=====================>........] - ETA: 7:12 - loss: 0.0396 - acc: 0.985 - ETA: 7:10 - loss: 0.0396 - acc: 0.985 - ETA: 7:09 - loss: 0.0398 - acc: 0.985 - ETA: 7:08 - loss: 0.0399 - acc: 0.985 - ETA: 7:06 - loss: 0.0400 - acc: 0.985 - ETA: 7:05 - loss: 0.0403 - acc: 0.984 - ETA: 7:04 - loss: 0.0405 - acc: 0.984 - ETA: 7:02 - loss: 0.0404 - acc: 0.984 - ETA: 7:01 - loss: 0.0405 - acc: 0.984 - ETA: 7:00 - loss: 0.0405 - acc: 0.984 - ETA: 6:58 - loss: 0.0406 - acc: 0.984 - ETA: 6:57 - loss: 0.0405 - acc: 0.984 - ETA: 6:56 - loss: 0.0404 - acc: 0.984 - ETA: 6:55 - loss: 0.0408 - acc: 0.984 - ETA: 6:53 - loss: 0.0407 - acc: 0.984 - ETA: 6:52 - loss: 0.0408 - acc: 0.984 - ETA: 6:51 - loss: 0.0407 - acc: 0.984 - ETA: 6:49 - loss: 0.0407 - acc: 0.984 - ETA: 6:48 - loss: 0.0410 - acc: 0.984 - ETA: 6:47 - loss: 0.0410 - acc: 0.984 - ETA: 6:45 - loss: 0.0409 - acc: 0.984 - ETA: 6:44 - loss: 0.0407 - acc: 0.984 - ETA: 6:43 - loss: 0.0408 - acc: 0.984 - ETA: 6:41 - loss: 0.0410 - acc: 0

34700/34700 [==============================] - ETA: 2:49 - loss: 0.0470 - acc: 0.982 - ETA: 2:48 - loss: 0.0469 - acc: 0.982 - ETA: 2:47 - loss: 0.0469 - acc: 0.982 - ETA: 2:45 - loss: 0.0468 - acc: 0.982 - ETA: 2:44 - loss: 0.0469 - acc: 0.982 - ETA: 2:43 - loss: 0.0471 - acc: 0.982 - ETA: 2:42 - loss: 0.0470 - acc: 0.982 - ETA: 2:40 - loss: 0.0472 - acc: 0.982 - ETA: 2:39 - loss: 0.0477 - acc: 0.982 - ETA: 2:38 - loss: 0.0477 - acc: 0.982 - ETA: 2:37 - loss: 0.0477 - acc: 0.982 - ETA: 2:35 - loss: 0.0477 - acc: 0.982 - ETA: 2:34 - loss: 0.0481 - acc: 0.982 - ETA: 2:33 - loss: 0.0480 - acc: 0.982 - ETA: 2:31 - loss: 0.0480 - acc: 0.982 - ETA: 2:30 - loss: 0.0480 - acc: 0.982 - ETA: 2:29 - loss: 0.0480 - acc: 0.982 - ETA: 2:28 - loss: 0.0480 - acc: 0.982 - ETA: 2:26 - loss: 0.0480 - acc: 0.982 - ETA: 2:25 - loss: 0.0480 - acc: 0.982 - ETA: 2:24 - loss: 0.0481 - acc: 0.982 - ETA: 2:22 - loss: 0.0481 - acc: 0.982 - ETA: 2:21 - loss: 0.0481 - acc: 0.982 - ETA: 2:20 - loss: 0.0482 - acc: 0

#finally we got training accuracy of 98 and test accuracy of 90.33